In [ ]:
!unzip /content/challenge-webmedia-e-globo-2023.zip

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [3]:
import glob
import os

In [4]:
user_path = "/content/drive/MyDrive/Faculdade/Pos_EngML/DataThon/datathon_files/files/treino"
site_path = "/content/drive/MyDrive/Faculdade/Pos_EngML/DataThon/datathon_files/itens/itens"

# Carregar dados

In [5]:
user_files = glob.glob(os.path.join(user_path, "*.csv"))
usuarios = pd.concat((pd.read_csv(f) for f in user_files), ignore_index=True)


In [ ]:
site_files = glob.glob(os.path.join(site_path, "*.csv"))
site = pd.concat((pd.read_csv(f) for f in site_files), ignore_index=True)

In [ ]:
validacao = pd.read_csv('/content/validacao.csv')

In [ ]:
# Pré-processamento dos dados de usuários
def parse_lista(valor):
    return [item.strip() for item in str(valor).split(',')]

def processar_usuarios(df):
    # Converter colunas de histórico em listas
    list_columns = ['history', 'timestampHistory', 'numberOfClicksHistory',
                   'timeOnPageHistory', 'scrollPercentageHistory', 'pageVisitsCountHistory']

    for col in list_columns:
        df[col] = df[col].apply(parse_lista)

    # Criar características agregadas
    df['total_cliques'] = df['numberOfClicksHistory'].apply(lambda x: sum(map(int, x)))
    df['tempo_medio'] = df['timeOnPageHistory'].apply(lambda x: np.mean(list(map(int, x))))
    df['scroll_medio'] = df['scrollPercentageHistory'].apply(lambda x: np.mean(list(map(float, x))))
    df['visitas_total'] = df['pageVisitsCountHistory'].apply(lambda x: sum(map(int, x)))

    return df

usuarios_processados = processar_usuarios(usuarios)

# Engenharia de características
features = usuarios_processados[['total_cliques', 'tempo_medio', 'scroll_medio', 'visitas_total']]
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Clusterização com KMeans
kmeans = KMeans(n_clusters=5, random_state=42)
usuarios_processados['cluster'] = kmeans.fit_predict(features_scaled)

In [ ]:
site

In [ ]:
!python3 -m spacy download pt_core_news_sm

In [ ]:
import spacy

nlp = spacy.load("pt_core_news_sm")
portuguese_stopwords = nlp.Defaults.stop_words

In [ ]:
# Pré-processamento de texto
site['conteudo'] = site['title'] + ' ' + site['caption'] + ' ' + site['body']
tfidf = TfidfVectorizer(stop_words=list(portuguese_stopwords))
tfidf_matrix = tfidf.fit_transform(site['conteudo'])

# Mapeamento de IDs para índices
indices = pd.Series(site.index, index=site['page']).drop_duplicates()

In [ ]:
class SistemaRecomendacao:
    def __init__(self, usuarios, site, clusters):
        self.usuarios = usuarios
        self.site = site
        self.clusters = clusters
        self.carregar_dados()

    def carregar_dados(self):
        # Popularidade por cluster
        self.popularidade_cluster = {}
        for cluster in self.clusters.unique():
            historico_cluster = self.usuarios[self.usuarios['cluster'] == cluster]['history'].explode()
            self.popularidade_cluster[cluster] = historico_cluster.value_counts().to_dict()

        # Matriz de similaridade
        self.cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

    def recomendar_por_cluster(self, cluster, n=10):
        return list(self.popularidade_cluster.get(cluster, {}).keys())[:n]

    def recomendar_por_conteudo(self, page_id, n=10):
        idx = indices.get(page_id)
        if idx is None: return []
        sim_scores = list(enumerate(self.cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_indices = [i[0] for i in sim_scores[1:n+1]]
        return self.site['Page'].iloc[sim_indices].tolist()

    def recomendar_hibrido(self, user_id, n=20):
        # Verificar se é usuário existente
        usuario = self.usuarios[self.usuarios['userId'] == user_id]

        if not usuario.empty:
            cluster = usuario['cluster'].values[0]
            historico = usuario['history'].values[0]
        else:
            cluster = np.random.choice(self.clusters.unique())
            historico = []

        # Recomendações baseadas em cluster
        rec_cluster = self.recomendar_por_cluster(cluster, n//2)

        # Recomendações baseadas em conteúdo
        rec_conteudo = []
        if historico:
            for artigo in historico[:3]:  # Considerar últimos 3 artigos
                rec_conteudo += self.recomendar_por_conteudo(artigo, 5)

        # Combinação e desduplicação
        recomendacoes = list(dict.fromkeys(rec_cluster + rec_conteudo))
        return recomendacoes[:n]

# Inicializar sistema
sistema = SistemaRecomendacao(usuarios_processados, site, usuarios_processados['cluster'])

In [ ]:
# Função para gerar recomendações para o conjunto de validação
def gerar_recomendacoes_validacao(df_validacao, sistema):
    recomendacoes = []
    for _, row in df_validacao.iterrows():
        user_id = row['userId']
        rec = sistema.recomendar_hibrido(user_id)
        recomendacoes.append({
            'userId': user_id,
            'recomendacoes': rec
        })
    return pd.DataFrame(recomendacoes)

# Gerar recomendações finais
recomendacoes_finais = gerar_recomendacoes_validacao(validacao, sistema)
print(recomendacoes_finais)

In [ ]:
# Atualizar popularidade considerando recência
def atualizar_recencia(site, peso_recencia=0.5):
    data_atual = pd.Timestamp.now()
    site['idade_dias'] = (data_atual - pd.to_datetime(site['issued'])).dt.days
    site['peso_recencia'] = 1 / (site['idade_dias'] + 1)
    return site

site = atualizar_recencia(site)

# Modificar a lógica de popularidade para incluir recência
def recomendar_por_cluster_com_recencia(self, cluster, n=10):
    artigos = self.popularidade_cluster.get(cluster, {})
    df = pd.DataFrame.from_dict(artigos, orient='index', columns=['contagem'])
    df = df.join(self.site.set_index('Page')[['peso_recencia']])
    df['score'] = df['contagem'] * df['peso_recencia']
    return df.sort_values('score', ascending=False).index.tolist()[:n]